In [1]:
%matplotlib inline

In [2]:
from Bio import SeqIO, Phylo
import glob
import random

import pandas as pd
import numpy as np

# Just working with OMA groups first

In [11]:
monophyly_df = pd.read_csv('../Data/OMA_group_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

(20, 9)


,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
OMAGroup_733878,IntNode_66,IntNode_91,0.52861,29.78150,55.18236,85.49247,203.0,92.0,111.0
OMAGroup_754919,IntNode_25,IntNode_90,0.12304,3.82319,1.20663,5.15286,204.0,92.0,112.0
OMAGroup_788814,IntNode_45,IntNode_53,1.15706,47.97359,29.46055,78.59120,200.0,78.0,122.0
OMAGroup_833097,IntNode_1,IntNode_90,0.19569,20.50439,27.28626,47.98634,214.0,91.0,123.0
OMAGroup_803985,IntNode_41,IntNode_53,2.17933,25.95126,32.27627,60.40686,165.0,54.0,111.0


In [65]:
target_mono_frac = 0.05
for tree_loc in glob.glob('../Data/OMA_group_data/eukaryotes/'
                          'processed_OMA_trees/*.NoZeroTerms.Rooted.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=10e-10)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = target_mono_frac*total_terms
        n_to_remove = round(len(mono_ids)-n_to_retain)
    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
        n_to_remove = round(len(other_ids)-n_to_retain)
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    ###I'm making this overly complicated here so that I don't ever accidentally remove the 
    ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
    ###removal of a single clade connected to the monophyletic node
    for i in range(n_to_remove):
        #######
        l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
        r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
        #######
        if len(l_terms) > 1 and len(r_terms) > 1:
            combined_terms = l_terms + r_terms
        elif len(l_terms) > 1 and len(r_terms) == 1:
            combined_terms = l_terms
        elif len(l_terms) == 1 and len(r_terms) > 1:
            combined_terms = r_terms
        else:
            print('Something unexpected is happening during node removal and you should investigate')
            print(len(l_terms), len(r_terms))
            break
        term_to_remove = random.choice(combined_terms)
        starting_tree.prune(term_to_remove)
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    assert initial_root_dist == final_root_dist
    outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    Phylo.write(starting_tree, outfile_loc, 'newick')

OMAGroup_659254.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_555739.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_555520.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_489052.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_733878.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_761172.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_498952.treefile.NoZeroTerms.Rooted.MPAJH
not mono
OMAGroup_833097.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_732528.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_803985.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_840944.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_788814.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_802797.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_819101.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_754919.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_786887.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_790712.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_837326.treefile.NoZeroTerms.Rooted.MPAJH
not mono
OMAGroup_665231.treefile.NoZeroTerms.Rooted.MPAJH
OMAGroup_830349.treefile.NoZeroT

# And a similar thing for the Tria *et al.* dataset 

In [66]:
monophyly_df = pd.read_csv('../Data/Tria_et_al_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

(200, 9)


,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
KOG0822,IntNode_12,IntNode_16,0.61014,11.20342,9.23987,21.05343,31.0,17.0,14.0
KOG1248,IntNode_13,IntNode_12,1.30570,6.10326,9.17116,16.58012,31.0,17.0,14.0
KOG3980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KOG0743,IntNode_12,IntNode_16,0.61684,7.44933,5.98348,14.04965,31.0,17.0,14.0
KOG3049,IntNode_5,IntNode_9,0.41951,4.51838,5.59950,10.53739,31.0,17.0,14.0


In [73]:
target_mono_frac = 0.1
for tree_loc in glob.glob('../Data/Tria_et_al_data/eukaryotes/'
                          'processed_trees/*.NoZeroTerms.Rooted.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=10e-10)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = target_mono_frac*total_terms
        n_to_remove = round(len(mono_ids)-n_to_retain)
    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
        n_to_remove = round(len(other_ids)-n_to_retain)
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    ###I'm making this overly complicated here so that I don't ever accidentally remove the 
    ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
    ###removal of a single clade connected to the monophyletic node
    for i in range(n_to_remove):
        #######
        l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
        r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
        #######
        if len(l_terms) > 1 and len(r_terms) > 1:
            combined_terms = l_terms + r_terms
        elif len(l_terms) > 1 and len(r_terms) == 1:
            combined_terms = l_terms
        elif len(l_terms) == 1 and len(r_terms) > 1:
            combined_terms = r_terms
        else:
            print('Something unexpected is happening during node removal and you should investigate')
            print(len(l_terms), len(r_terms))
            break
        term_to_remove = random.choice(combined_terms)
        starting_tree.prune(term_to_remove)
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    assert initial_root_dist == final_root_dist
    outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
#     Phylo.write(starting_tree, outfile_loc, 'newick')

KOG4067.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG3181.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG0385.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG4077.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG1287.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG0258.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG2179.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG0187.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG1158.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG1248.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG2393.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG3083.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG3169.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG1128.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG4017.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
0.17647058823529413
KOG3291.faa.aln.nwk.NoZer

ValueError: target 'I' is not in this tree

In [74]:
2/35

0.05714285714285714